# MonoReader RNN

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, Subset
import matplotlib.pyplot as plt
import time
import os
import copy
import glob
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import pandas as pd




cudnn.benchmark = True

## 1. Data Loading and Initialization

In [2]:
num_frames = 4

h, w =224, 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [3]:
from PIL import Image
import random

def get_images(path2ajpgs):
    listOfClasses = os.listdir(path2ajpgs)
    ids = []
    labels = []
    for catg in listOfClasses:
        path2catg = os.path.join(path2ajpgs, catg)
        listOfSubCats = os.listdir(path2catg)
        path2subCats= [os.path.join(path2catg,los) for los in listOfSubCats]
        ids.extend(path2subCats)
        labels.extend([catg]*len(listOfSubCats))
    return ids, labels, listOfClasses 

class VideoDataset(Dataset):
    def __init__(self, ids, labels, transform):      
        self.transform = transform
        self.ids = ids
        self.labels = labels
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        path2imgs=glob.glob(self.ids[idx]+"/*.jpg")
        #path2imgs = path2imgs[:num_frames]
        label = labels_dict[self.labels[idx]]
        frames = []
        for p2i in path2imgs:
            frame = Image.open(p2i)
            frames.append(frame)
        seed = np.random.randint(1e9)        
        frames_tr = []
        for frame in frames:
            random.seed(seed)
            np.random.seed(seed)
            frame = self.transform(frame)
            frames_tr.append(frame)
        if len(frames_tr)>0:
            frames_tr = torch.stack(frames_tr)
        return frames_tr, label

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
training_dir = os.path.join('MonReader_images_Rnn', 'training')

train_ids,train_labels,catgs = get_images(training_dir)

train_transformer = transforms.Compose([
            transforms.Resize((h,w)),
            transforms.RandomHorizontalFlip(p=0.5),  
            transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),    
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            ])   

In [6]:
labels_dict = {}
ind = 0
for uc in catgs:
    labels_dict[uc] = ind
    ind+=1
labels_dict 

{'flip': 0, 'notflip': 1}

In [7]:
train_ds = VideoDataset(ids= train_ids, labels= train_labels, transform= train_transformer)
print(len(train_ds))


117


In [8]:
imgs, label = train_ds[10]
imgs.shape, label, torch.min(imgs), torch.max(imgs)

(torch.Size([21, 3, 224, 224]), 0, tensor(-2.1179), tensor(2.1694))

In [9]:
testing_dir = os.path.join('MonReader_images_Rnn', 'testing')

test_ids,test_labels,catgs = get_images(testing_dir)

test_transformer = transforms.Compose([
            transforms.Resize((h,w)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            ]) 

In [10]:

test_ds = VideoDataset(ids= test_ids, labels= test_labels, transform= test_transformer)
print(len(test_ds))

115


In [11]:
dataset_sizes=dict()
dataset_sizes['training']=len(train_ds)
dataset_sizes['testing']=len(test_ds)
print(dataset_sizes)

{'training': 117, 'testing': 115}


In [12]:
def collate_fn_rnn(batch):
    imgs_batch, label_batch = list(zip(*batch))
    imgs_batch = [imgs for imgs in imgs_batch if len(imgs)>0]
    label_batch = [torch.tensor(l) for l, imgs in zip(label_batch, imgs_batch) if len(imgs)>0]
    imgs_tensor = torch.stack(imgs_batch)
    labels_tensor = torch.stack(label_batch)
    return imgs_tensor,labels_tensor
train_dl = torch.utils.data.DataLoader(train_ds, batch_size= 1,
                        shuffle=True, collate_fn= collate_fn_rnn)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size= 1,
                        shuffle=False, collate_fn= collate_fn_rnn)  

In [13]:
for xb,yb in train_dl:
    print(xb.shape, yb.shape)
    break

torch.Size([1, 16, 3, 224, 224]) torch.Size([1])


In [14]:
for xb,yb in test_dl:
    print(xb.shape, yb.shape)
    break

torch.Size([1, 1, 3, 224, 224]) torch.Size([1])


## 2. Rnn Model

In [15]:
from torch import nn
class Resnt18Rnn(nn.Module):
    def __init__(self, params_model):
        super(Resnt18Rnn, self).__init__()
        num_classes = params_model["num_classes"]
        dr_rate= params_model["dr_rate"]
        pretrained = params_model["pretrained"]
        rnn_hidden_size = params_model["rnn_hidden_size"]
        rnn_num_layers = params_model["rnn_num_layers"]
        
        baseModel = models.resnet18(pretrained=pretrained)
        num_features = baseModel.fc.in_features
        baseModel.fc = Identity()
        self.baseModel = baseModel
        self.dropout= nn.Dropout(dr_rate)
        self.rnn = nn.LSTM(num_features, rnn_hidden_size, rnn_num_layers)
        self.fc1 = nn.Linear(rnn_hidden_size, num_classes)
    def forward(self, x):
        b_z, ts, c, h, w = x.shape
        ii = 0
        y = self.baseModel((x[:,ii]))
        out, (hn, cn) = self.rnn(y.unsqueeze(1))
        for ii in range(1, ts):
            y = self.baseModel((x[:,ii]))
            out, (hn, cn) = self.rnn(y.unsqueeze(1), (hn, cn))
        out = self.dropout(out[:,-1])
        out = self.fc1(out) 
        return out 
    
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x   

### 2.1. Model Training

In [16]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['training', 'testing']:
            if phase == 'training':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0


            dataloader = train_dl
            if phase == 'testing':
                dataloader = test_dl
            # Iterate over data.
            
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'training':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'training':
                scheduler.step(running_loss / dataset_sizes[phase])

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'testing' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [17]:
params_model={
        "num_classes": 2,
        "dr_rate": 0.1,
        "pretrained" : True,
        "rnn_num_layers": 1,
        "rnn_hidden_size": 100,}
model = Resnt18Rnn(params_model)
model = model.to(device)

loss_func = nn.CrossEntropyLoss(reduction="sum")
opt = optim.Adam(model.parameters(), lr=3e-5)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=5,verbose=1)

In [18]:
model = train_model(model, loss_func, opt, lr_scheduler,
                       num_epochs=2)

Epoch 0/1
----------
training Loss: 0.6978 Acc: 0.4872
testing Loss: 0.6919 Acc: 0.5217

Epoch 1/1
----------
training Loss: 0.5944 Acc: 0.8632
testing Loss: 0.7926 Acc: 0.3826

Training complete in 5m 39s
Best val Acc: 0.521739


### 2.2. Model Evalution

In [19]:
was_training = model.training
model.eval()

y_train=[]
y_train_pred=[]
with torch.no_grad():
    for i, (inputs, labels) in enumerate(torch.utils.data.DataLoader(train_ds)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        y_train.append(catgs[labels])
        y_train_pred.append(catgs[preds])
    y_train=pd.DataFrame(data=y_train)
    y_train_pred=pd.DataFrame(data=y_train_pred)

    model.train(mode=was_training)

In [20]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import fbeta_score

print('Train accuracy' ,accuracy_score(y_train,y_train_pred))

print(classification_report(y_train,y_train_pred))
pd.crosstab(y_train[0],y_train_pred[0])

Train accuracy 0.47863247863247865
              precision    recall  f1-score   support

        flip       0.54      0.45      0.49        65
     notflip       0.43      0.52      0.47        52

    accuracy                           0.48       117
   macro avg       0.48      0.48      0.48       117
weighted avg       0.49      0.48      0.48       117



,flip,notflip
0,,
flip,29,36
notflip,25,27


In [21]:
was_training = model.training
model.eval()

y_test=[]
y_test_pred=[]
with torch.no_grad():
    for i, (inputs, labels) in enumerate(torch.utils.data.DataLoader(test_ds)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        y_test.append(catgs[labels])
        y_test_pred.append(catgs[preds])
    y_test=pd.DataFrame(data=y_test)
    y_test_pred=pd.DataFrame(data=y_test_pred)

    model.train(mode=was_training)

In [22]:
print('Test accuracy' ,accuracy_score(y_test,y_test_pred))

print('Test F1' ,fbeta_score(y_test, y_test_pred, average='macro', beta=1))

print(classification_report(y_test,y_test_pred))
pd.crosstab(y_test[0],y_test_pred[0])

Test accuracy 0.5217391304347826
Test F1 0.5180952380952382
              precision    recall  f1-score   support

        flip       0.60      0.40      0.48        63
     notflip       0.48      0.67      0.56        52

    accuracy                           0.52       115
   macro avg       0.54      0.53      0.52       115
weighted avg       0.54      0.52      0.51       115



,flip,notflip
0,,
flip,25,38
notflip,17,35
